<h3>Módulo para aplicação de algoritmos em base dividida em treino e teste</h3>
Importação das bibliotecas necessárias

In [ ]:
"""
EDUARDO BISPO FELIZARDO | 10400894
GUSTAVO AUGUSTO ALVES PIVATTO | 10401400
JEFFERSON DA CONCEICAO DANTAS | 10401327
MARINA MIKI SINZATO | 10401880 

Objetivo do Notebook
O notebook tem como foco a aplicação de modelos preditivos em uma base de dados de vendas de café, 
com o objetivo de comparar seus desempenhos usando o erro percentual absoluto médio (MAPE).

Bibliotecas Importadas
O código importa diversas bibliotecas para manipulação de dados, modelagem e visualização, incluindo:

pandas, numpy, matplotlib
sklearn.metrics para cálculo do MAPE
statsmodels para o modelo Holt-Winters (ExponentialSmoothing)
xgboost para XGBRegressor
pmdarima para uso do AutoARIMA
train_test_split para divisão entre treino e teste

25/03/2025,EDUARDO BISPO FELIZARDO,Importação de bibliotecas e definição do ambiente de trabalho.
25/03/2025,GUSTAVO AUGUSTO ALVES PIVATTO,Leitura da base de dados tratada.
26/03/2025,JEFFERSON DA CONCEICAO DANTAS,Divisão da base em treino e teste.
27/03/2025,MARINA MIKI SINZATO,Aplicação do modelo Holt-Winters (Exponential Smoothing).
27/03/2025,MARINA MIKI SINZATO,Aplicação do modelo XGBoost Regressor.
28/03/2025,GUSTAVO AUGUSTO ALVES PIVATTO,Aplicação do modelo AutoARIMA.
30/03/2025,JEFFERSON DA CONCEICAO DANTAS,Cálculo e comparação dos valores de MAPE entre os modelos.

"""

In [41]:
import pandas as pd
from pathlib import Path

import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np

from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.model_selection import train_test_split

from xgboost import XGBRegressor
from pmdarima import auto_arima

import warnings

<b>Carregamento de arquivos</b>
<br>
Define o caminho do arquivo que contém os dados trataods, que é gerado após rodar o arquivo 2_tratamento_dados.ipynb

In [42]:
caminho_arquivo_base = Path().resolve()  / "Base de Dados/Base_Atualizada.csv"

In [43]:
df_base = pd.read_csv(caminho_arquivo_base, sep=";")

In [44]:
df_base['date'] = pd.to_datetime(df_base['date'], format='%d/%m/%Y')
df_base = df_base.sort_values(by='date')

<b>Criação de funções para treinar modelos de previsão de séries temporais usando Holt-Winters (Exponential Smoothing), XGBoost e Auto-ARIMA.</b>

<b>Média Móvel:</b> Técnica de suavização de dados que calcula a média de um conjunto de valores dentro de uma janela móvel ao longo do tempo

In [45]:
def forecast_media_movel(df_cafe, train_size, forecast_size):
    mape_scores_ma = []

    for i in range(0, len(df_cafe) - train_size - forecast_size + 1):
        train_set = df_cafe["sales"][i:i + train_size]
        test_set = df_cafe["sales"][i + train_size:i + train_size + forecast_size]

        df_cafe["media_movel"] = df_cafe["sales"].rolling(window=len(train_set)).mean()
        media_movel_forecast = df_cafe["media_movel"].iloc[-len(test_set):]

        mape_ma = mean_absolute_percentage_error(test_set, media_movel_forecast)
        mape_scores_ma.append(mape_ma)

    return mape_scores_ma

<b>Holt-Winters: </b> Modelo de suavização exponencial que considera tendência e sazonalidade em séries temporais. Ele possui três componentes: nível (média ajustada), tendência (crescimento ou declínio) e sazonalidade (padrões repetitivos)

In [46]:
def forecast_holtwinters(df_cafe, train_size, forecast_size):
    df_cafe['sales'] = df_cafe['sales'].apply(lambda x: x if x > 0 else 0.01)    
    mape_scores_hw = []
    
    grid_params = {
        'trend': ['None','add', 'mul'],
        'seasonal': ['add', 'mul'],
        'seasonal_periods': [1,3,4]
    }
    
    best_params_hw = None 
    
    for i in range(0, len(df_cafe) - train_size - forecast_size + 1):
        train_set = df_cafe['sales'][i:i + train_size]
        test_set = df_cafe['sales'][i + train_size:i + train_size + forecast_size]
        
        best_mape = float('inf')
        best_params = None
        
        for trend in grid_params['trend']:
            for seasonal in grid_params['seasonal']:
                for seasonal_periods in grid_params['seasonal_periods']:
                    try:
                        model = ExponentialSmoothing(
                            train_set,
                            trend=trend,
                            seasonal=seasonal,
                            seasonal_periods=seasonal_periods,
                            use_boxcox=True
                        )
                        fit_model = model.fit()
                        forecast = fit_model.forecast(len(test_set))
                        
                        mape = mean_absolute_percentage_error(test_set, forecast)
                        
                        if mape < best_mape:
                            best_mape = mape
                            best_params = (trend, seasonal, seasonal_periods)  
                    except:
                        continue
        
        mape_scores_hw.append(best_mape)
        if best_params is not None:
            best_params_hw = best_params 
    
    return mape_scores_hw, best_params_hw 

<b>AutoARIMA: </b>Técnica automatizada para identificar o modelo ARIMA (AutoRegressive Integrated Moving Average) mais adequado para uma série temporal. O ARIMA é um modelo amplamente utilizado em séries temporais, composto por três componentes: autoregressivo (AR), de média móvel (MA) e de diferenciação (I), que juntos ajudam a modelar as dependências temporais. O AutoARIMA automatiza o processo de seleção desses parâmetros

In [47]:
def forecast_arima(df_cafe, train_size, forecast_size):

    mape_scores_arima = []

    for i in range(0, len(df_cafe) - train_size - forecast_size + 1):
        train_set = df_cafe['sales'][i:i + train_size]
        test_set = df_cafe['sales'][i + train_size:i + train_size + forecast_size]

        # Ajuste do modelo ARIMA
        model_auto_arima = auto_arima(train_set,
                                    start_p=0,
                                    start_q=0,
                                    start_d=0,
                                    max_p=6,
                                    max_d=1,
                                    max_q=6,
                                    seasonal=False,
                                    trace=False,
                                    suppress_warnings=True,
                                    stepwise=True,
                                    max_iter=50)

        arima_forecast = model_auto_arima.predict(n_periods=len(test_set))
        
        arima_forecast = arima_forecast[:len(test_set)].reset_index(drop=True)
        test_set = test_set.reset_index(drop=True)

        filtro = test_set > 0
        test_set = test_set[filtro].reset_index(drop=True)
        arima_forecast = arima_forecast[filtro].reset_index(drop=True)

        mape_arima = mean_absolute_percentage_error(test_set, arima_forecast)
        mape_scores_arima.append(mape_arima)

    return mape_scores_arima

<b>XGBoost com Lags: </b>O uso de lags no XGBoost em séries temporais envolve criar variáveis que representam os valores de uma série em pontos anteriores no tempo, ajudando o modelo a capturar dependências temporais. Essas variáveis de lag são usadas como entradas no modelo, permitindo que o XGBoost preveja o valor futuro com base nas observações passadas, o que é essencial para capturar padrões e tendências temporais em dados.

In [ ]:
def create_lag_features(df_cafe, lags):
    df_lagged = df_cafe.copy()
    
    for lag in range(1, lags + 1):
        df_lagged[f'lag_{lag}'] = df_lagged['sales'].shift(lag)
    
    df_lagged = df_lagged.dropna().reset_index(drop=True)  
    
    return df_lagged

In [49]:
def forecast_xgboost_lags(df_cafe, train_size, forecast_size):
    df_lagged = create_lag_features(df_cafe[['date', 'sales']], lags=6)
    X = df_lagged.drop(columns=['date', 'sales'])
    y = df_lagged['sales']
    
    mape_scores_lags = []

    for i in range(0, len(df_lagged) - train_size - forecast_size + 1):
        X_train = X.iloc[i:i + train_size]
        y_train = y.iloc[i:i + train_size]
        X_test = X.iloc[i + train_size:i + train_size + forecast_size]
        y_test = y.iloc[i + train_size:i + train_size + forecast_size]

        # Ajuste do modelo XGBoost
        model = XGBRegressor(random_state=42, 
                             n_estimators=1500, 
                             learning_rate=0.05, 
                             subsample=0.85,
                             max_depth=4, 
                             reg_alpha=0.05, 
                             reg_lambda=1, 
                             objective='reg:squarederror'
                             )
        
        model.fit(X_train, y_train)
        xgboost_forecast = model.predict(X_test)

        mape_lags = mean_absolute_percentage_error(y_test, xgboost_forecast)
        mape_scores_lags.append(mape_lags)
    
    return mape_scores_lags

In [50]:
min_dados = 24
cafes_com_dados = df_base.groupby('coffee_name').size()

cafes_filtrados = cafes_com_dados[cafes_com_dados >= min_dados].index

In [51]:
cafes_filtrados

Index(['Americano', 'Americano with Milk', 'Cappuccino', 'Cocoa', 'Cortado',
       'Espresso', 'Hot Chocolate', 'Latte'],
      dtype='object', name='coffee_name')

<b>Criação de Loop para iterar por todos os tipos de café</b><br>
Calculando e exibindo o valor do MAPE (Mean Absolute Percentage Error) para cada tipo de café.

In [ ]:
warnings.filterwarnings('ignore')

train_size = 6 
forecast_size = 1
  
resultados = []

for cafe in cafes_filtrados:
    df_cafe = df_base[df_base["coffee_name"] == cafe]

    df_cafe = df_cafe.groupby(df_cafe["date"].dt.to_period("M")).size().reset_index(name="sales")
        
    mape_holtwinters, best_params_hw = forecast_holtwinters(df_cafe, train_size, forecast_size)
    mape_media_movel = forecast_media_movel(df_cafe, train_size, forecast_size)
    mape_xgboost_lags = forecast_xgboost_lags(df_cafe, train_size, forecast_size)
    mape_arima = forecast_arima(df_cafe, train_size, forecast_size)

    resultados.append((
        cafe, 
        "holtwinters", 
        np.mean(mape_holtwinters), 
        best_params_hw[0] if best_params_hw else None,
        best_params_hw[1] if best_params_hw else None,
        best_params_hw[2] if best_params_hw else None
    ))

    resultados.append((
        cafe, 
        "media_movel", 
        np.mean(mape_media_movel),
        None, 
        None, 
        None
    ))

    resultados.append((
        cafe, 
        "auto_arima", 
        np.mean(mape_arima),
        None, 
        None, 
        None
    ))

    resultados.append((
        cafe, 
        "xgboost", 
        np.mean(mape_xgboost_lags),
        None, 
        None, 
        None
    ))

In [53]:
df_resultados = pd.DataFrame(resultados, columns=["coffee_name", "modelo", "MAPE", "trend", "seasonal", "seasonal_periods"])
df_resultados.to_csv('Bases Geradas/resultados_mape.csv', index=False)